In [13]:
# %pip install pytorch_lightning

In [14]:
# imports
import torch
import torch.nn as nn
import torch.optim as optim
import math
import pytorch_lightning as pl
import torchaudio

In [15]:
torch.cuda.is_available()

False

In [16]:
class SpeechEmotionRecognitionModel(pl.LightningModule):
    def __init__(self, input_size, num_classes, dim_model=512, nhead=8, num_encoder_layers=6, num_decoder_layers=6, lr=0.001):
        super(SpeechEmotionRecognitionModel, self).__init__()
        self.lr = lr
        self.transformer = nn.Transformer(d_model=dim_model, nhead=nhead,
                                          num_encoder_layers=num_encoder_layers,
                                          num_decoder_layers=num_decoder_layers)
        self.encoder = nn.Linear(input_size, dim_model)
        self.decoder = nn.Linear(dim_model, num_classes)
        self.loss_function = nn.CrossEntropyLoss()

    def forward(self, src):
        src = self.encoder(src)
        output = self.transformer(src, src)
        output = self.decoder(output)
        return output

    def training_step(self, batch, batch_idx):
        src, tgt = batch
        output = self(src)
        loss = self.loss_function(output, tgt)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

In [17]:
from local_dataset import AudioEmotionsDataset 
# import TQDMProgressBar
from pytorch_lightning.callbacks import TQDMProgressBar

In [18]:
dataset = AudioEmotionsDataset(max_size=100, batch_size=4)

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

max_len = 80080


In [19]:
model = SpeechEmotionRecognitionModel(input_size=dataset.feature_count, num_classes=dataset.class_count)

/Users/amittaijoel/miniconda3/envs/ling-project/lib/python3.11/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [20]:
dataset.feature_count

80080

In [21]:
dataset.class_count

7

In [24]:
model = SpeechEmotionRecognitionModel(input_size=dataset.feature_count, num_classes=7)

trainer = pl.Trainer(callbacks=[TQDMProgressBar(refresh_rate=10)], accelerator="auto", max_epochs=50, min_epochs=10)
trainer.fit(model, dataset.train_dataloader, dataset.test_dataloader)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type             | Params
---------------------------------------------------
0 | transformer   | Transformer      | 44.1 M
1 | encoder       | Linear           | 41.0 M
2 | decoder       | Linear           | 3.6 K 
3 | loss_function | CrossEntropyLoss | 0     
---------------------------------------------------
85.1 M    Trainable params
0         Non-trainable params
85.1 M    Total params
340.582   Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

In [ ]:
# print lengths of train features

for batch in dataset.train_dataloader:
    for i, features in enumerate(batch[0]):
        print(f"{features.shape}: {batch[1][i]}")

torch.Size([80080]): tensor([0., 1., 0., 0., 0., 0., 0.])
torch.Size([80080]): tensor([0., 0., 0., 1., 0., 0., 0.])
torch.Size([80080]): tensor([1., 0., 0., 0., 0., 0., 0.])
torch.Size([80080]): tensor([0., 0., 0., 0., 0., 1., 0.])
torch.Size([80080]): tensor([0., 0., 0., 1., 0., 0., 0.])
torch.Size([80080]): tensor([0., 0., 0., 0., 0., 1., 0.])
torch.Size([80080]): tensor([0., 0., 1., 0., 0., 0., 0.])
torch.Size([80080]): tensor([0., 0., 0., 0., 0., 1., 0.])
torch.Size([80080]): tensor([0., 0., 1., 0., 0., 0., 0.])
torch.Size([80080]): tensor([0., 1., 0., 0., 0., 0., 0.])
torch.Size([80080]): tensor([1., 0., 0., 0., 0., 0., 0.])
torch.Size([80080]): tensor([0., 1., 0., 0., 0., 0., 0.])
torch.Size([80080]): tensor([0., 1., 0., 0., 0., 0., 0.])
torch.Size([80080]): tensor([0., 0., 0., 0., 1., 0., 0.])
torch.Size([80080]): tensor([0., 0., 0., 1., 0., 0., 0.])
torch.Size([80080]): tensor([0., 1., 0., 0., 0., 0., 0.])
torch.Size([80080]): tensor([0., 0., 0., 0., 1., 0., 0.])
torch.Size([80

In [ ]:
import torch.nn.functional as F
def one_hot_encode(y):
    """
        One-hot encode a single datapoint.
        (Maybe do this all at once for the entire dataset?)
    """
    encoded = F.one_hot(torch.tensor(y), num_classes=7)
    print(f"{encoded = }")
    # change to float
    encoded = encoded.float()
    return encoded

one_hot_encode(3)

encoded = tensor([0, 0, 0, 1, 0, 0, 0])


tensor([0., 0., 0., 1., 0., 0., 0.])